<a href="https://colab.research.google.com/github/AyozeGS/IABD/blob/main/7RO/T5/7RO_3_en_raya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENTRENANDO A UN AGENTE PARA JUGAR AL TRES EN RAYA MEDIANTE Q-LEARNING

Se implementará el algoritmo Q-Learning para que un agente aprenda a jugar al tres en raya.

Se evaluará el desempeño del agente analizando el porcentaje de partidas ganadas frente a jugador aleatorio y experto.

Enlace al cuaderno:

https://colab.research.google.com/drive/1I7VZcHsz2qYdlCaKmjscKRjEqP1SmdVo#scrollTo=HqMl35xfE547

# Planificación

In [ ]:
#Librerías
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

## Preparación

### Tableros válidos y matriz de recompensas

Se comprueban todas las combinaciones de 0,1,2 en nueve casillas, representando los tableros posibles y los válidos una vez eliminados aquellas con más 2 que 1 o con dos ó más 1 que 2.

Además se crea un vector con las recompensas de victoria/derrota/empate para cada combinación válida y según el resultado.

Cada fila representaría una tablero posible, recorrido de izquierda a derecha y de arriba hacia abajo, con 'x' como 2, 'o' como 1 y los restantes como 0. Un ejemplo sería:

|   |   |   |
|---|---|---|
| X |   | O  |
| O | X | X |
| X |   | O |

[1, 0, 2, 2, 1, 1, 1, 0, 2]



In [ ]:
# Vectores de tableros válidos y matriz de recompensas
def create_board_and_rewards_vectors(rewards, info=False):

  #Matriz de tableros válidos y array de victoria
  board_states = []
  v_wins = np.ones(3)

  #Variables de recompensas
  R = []
  r_win = rewards[0]
  r_lose = rewards[1]
  r_draw = rewards[2]

  #variables auxiliares para información
  count = 0
  valid_count = 0
  winner_counts = [0,0,0] #Win, Lose, Draw

  #Combinatoria de 3 (0,1 y 2) posibles valores en 9 casillas
  for a in range(3):
    for b in range(3):
      for c in range(3):
        for d in range(3):
          for e in range(3):
            for f in range(3):
              for g in range(3):
                for h in range(3):
                  for i in range(3):

                    count +=1 #contador de combinaciones
                    board_state = [a, b, c, d, e, f, g, h, i] #vector representación del tablero

                    #Se omite el tablero si J1 no tiene igual cantidad de fichas que J2 o una más.
                    diff_pieces = board_state.count(1) - board_state.count(2)
                    if diff_pieces > 1 or diff_pieces < 0:
                      continue

                    #Se detectan 3 en raya de ambos jugadores en el tablero
                    players_win = [False, False]
                    m = np.asmatrix(np.array(board_state).reshape(3,3))
                    #Diagonales
                    diag1 = m.diagonal()
                    diag2 = np.fliplr(m).diagonal()

                    for i in range(1,3):
                      #Jugador i gana en filas o columnas
                      if np.any(np.all(i*v_wins == m, axis=0)) or np.any(np.all(i*v_wins == m, axis=1)):
                        players_win[i-1] = True
                      #Jugador i gana en diagonales
                      if np.all(i*v_wins == diag1) or np.all(i*v_wins == diag2):
                        players_win[i-1] = True
                    #Si ambos han ganado se omite el tablero
                    if all(players_win):
                      continue
                    #Se añade el tablero al array de tableros válidos
                    board_states.append(board_state)

                    #Se añade la recompensa del tablero al vector de recompensas
                    if players_win[0]:
                      R.append(r_win)
                      winner_counts[0] += 1
                    elif players_win[1]:
                      R.append(r_lose)
                      winner_counts[1] += 1
                    elif board_state.count(0) == 0: #Si el tablero está lleno y no se hay ganador
                      R.append(r_draw)
                      winner_counts[2] += 1
                    else:
                      R.append(0)

                    valid_count += 1 #contador de estados
  #INFO
  if info:
    print(f"{valid_count} Estados Validos de {count} combinaciones")
    print(f"Estados Finales: {winner_counts[0]} V / {winner_counts[1]} D / {winner_counts[2]} E")

  return board_states, R

## LLamada de la función
# Recompensas de victoria/derrota/empate
_rewards = [10, -10, 1]
_board_states, _R = create_board_and_rewards_vectors(_rewards, True)

5890 Estados Validos de 19683 combinaciones
Estados Finales: 942 V / 412 D / 16 E


In [ ]:
winner_counts = [0,0,0] #Win, Lose, Draw
winner_counts[1] += 1
winner_counts

[0, 1, 0]

### Diccionarios

Diccionarios directo e inverso con valores (representación numérica del tablero) y estados(índices) de cada tablero

el valor se cálcula de cada tablero tal que:

$$ v = a \cdot 3^8 + b \cdot 3^7 + c \cdot 3^6 + d \cdot 3^5 + e \cdot 3^4 + f \cdot 3^3 + g \cdot 3^2 + h \cdot 3^1 + i \cdot 3^0 $$

siendo los coeficientes en el rango a-i los valores del array para el tablero en orden recorrido.

In [ ]:
def create_dicts_state_value(board_states, info=False):
  #Diccionarios
  dict_state_value = {}
  dict_value_state = {}

  #Se rellenan ambos diccionarios
  for index, board_state in enumerate(board_states):
    board_value = sum([board_state[i] * 3**(8-i) for i in range(9)])
    dict_value_state[board_value] = index
    dict_state_value[index] = board_value

  #INFO
  if info:
    print("E -> V / V -> E")
    for i in range(10):
      print([*dict_state_value.keys()][i], "->", [*dict_state_value.values()][i], "/",
            [*dict_value_state.keys()][i], "->", [*dict_value_state.values()][i])

  return dict_state_value, dict_value_state

## LLamada de la función
_dict_state_value, _dict_value_state = create_dicts_state_value(_board_states, True)

E -> V / V -> E
0 -> 0 / 0 -> 0
1 -> 1 / 1 -> 1
2 -> 3 / 3 -> 2
3 -> 5 / 5 -> 3
4 -> 7 / 7 -> 4
5 -> 9 / 9 -> 5
6 -> 11 / 11 -> 6
7 -> 14 / 14 -> 7
8 -> 15 / 15 -> 8
9 -> 16 / 16 -> 9


### Matriz de Transición

Matriz que para cada estado indica los estados a los que se transitaría con cada acción realizada. En casa de ser una acción no válida se añadiría menos uno.

Por ejemplo, partiendo del estado 1701, cuyo tablero es:

|   |   |   |
|---|---|---|
|   | O |   |
| O | O | X |
| X | X |   |

1701 => [0, 2, 0, 2, 2, 1, 1, 1, 0] => V 5061

La matriz de transición en dicha fila pondría las posiciones ocupadas a -1.

T[1701] -> [-, -1, -, -1, -1, -1, -1, -1, -]

Y para las tres posiciones restante calcularía que el valor actual del tablero es 5061 (usando la fóruma vista antes) y le aplicaría el valor del nuevo movimiento. Por ejemplo en la última casilla sería 1 * 3^0 = 1, quedando el tablero a 5062. Con dicho valor se busca en el diccionario inverso que corresponde al estado 1702 y ese valor se añade a la matriz de transicción para esa fila y en la columna designada para la acción 8 (mover en la última casilla).

|   |   |   |
|---|---|---|
|   | O |   |
| O | O | X |
| X | X | X |

[0, 2, 0, 2, 2, 1, 1, 1, 1] => V 5062 => 1702

De ese modo, si hacemos el cálculo para los otros dos movimientos posibles en las casillas superiores la matriz de transición en esa fila quedaría de la forma

[3685,   -1, 1960,   -1,   -1,   -1,   -1,   -1, 1702]

In [ ]:
def create_transition_matrix(board_states, R, dict_state_value, dict_value_state, info=False):
  # Matriz de Transición
  T = []
  for index, board_state in enumerate(board_states):

    # Si el estado es un estado final se añade array de nueve -1
    if R[index] != 0:
      T.append([-1]*9)
      continue

    # Si juega J1(igualdad de fichas) se añade un 1  y si juega J2 se añade un 2
    piece = 1 if board_state.count(1) == board_state.count(2) else 2

    # Se crea una nueva fila a añadir a la matriz de transiciones
    T_row = []
    for position in range(9):
      # Se calcula estado siguiente en cada posicion de añadir 1/2 a la casilla
      if board_state[position] == 0:
        T_row.append(dict_value_state[dict_state_value[index] + piece*3**(8-position)])
      # Se añade -1 a la transición si ya había un 1/2 en la casilla
      else:
        T_row.append(-1)
    T.append(T_row)
  T = np.array(T, dtype=int)

  # INFO
  if info:
    print("Primeras filas de la matriz T:\n", T[0:5])

  return T

# LLamada de la función
_T = create_transition_matrix(_board_states, _R, _dict_state_value, _dict_value_state, True)

Primeras filas de la matriz T:
 [[2103  746  265   96   35   13    5    2    1]
 [4154 1487  530  191   70   26   10    4   -1]
 [4155 1488  531  192   71   27   11   -1    3]
 [2105  748  267   98   37   15    7   -1   -1]
 [2107  750  269  100   39   17    9   -1   -1]]


## Entrenamiento

### Funciones Auxiliares

Función que obtiene los índices de la matriz de transición para un estado donde el valor no es -1, que corresponderían a jugadas válidas, y devuelve uno al azar.

In [ ]:
def choose_random_action(state, T):

  indexes = np.where(T[state] != -1)[0]
  try:
    return np.random.choice(indexes)
  except ValueError:
    print(f"Warning: 'choose_random_action'! No hay transiciones válidas para el estado {state}. 'None' devuelto.")

#Examples
T_example=np.zeros((4,9),dtype=float)
for i in range(T_example.shape[0]):
  for j in range(T_example.shape[1]):
    T_example[i,j] = np.random.randint(10)
  if i == 2:
    T_example[i] = [-1]*T_example.shape[1]

print("Ejemplo 1:")
print("T:",T_example[2])
print(choose_random_action(2, T_example))
print("Ejemplo 2:")
print("T:",_T[0])
for i in range(5):
  print(choose_random_action(0, T_example))

Ejemplo 1:
T: [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
None
Ejemplo 2:
T: [2103  746  265   96   35   13    5    2    1]
0
2
0
2
3


Función que calcula el máximo valor de una fila de la matriz Q, ignorando aquellos valores que no sean válidos (-1) según la matriz T para el mismo estado.

In [ ]:
def calculate_q_max(state, T, Q, ignore_invalid_state=False):

  T_mask = T[state]==-1
  if T_mask.all() and ignore_invalid_state:
    return 0
  else:
    return np.ma.max(np.ma.masked_array(Q[state], T_mask))

Función que obtiene los índices de la matriz de transición donde el valor es igual al máximo de la matriz Q para un estado dado.

In [ ]:
def choose_best_action(state, T, Q):

  q_value = calculate_q_max(state, T, Q, False)
  indexes = np.where(Q[state,] == q_value)[0]
  try:
    return np.random.choice(indexes)
  except ValueError:
    print(f"Warning: 'choose_best_action'! No hay transiciones válidas para el estado {state}. 'None' devuelto.")

#Examples
T_example=np.zeros((4,9),dtype=float)
Q_example=np.zeros((4,9),dtype=float)
for i in range(T_example.shape[0]):
  for j in range(T_example.shape[1]):
    T_example[i,j] = np.random.randint(10)
    T_example[i,j] = np.random.choice([T_example[i,j], -1])
    Q_example[i,j] = np.random.random()
  if i == 2:
    T_example[i] = [-1]*T_example.shape[1]

print("Ejemplo 1:")
print("T:",T_example[2])
print("Q:",Q_example[2])
print(choose_best_action(2, T_example, Q_example))
print("Ejemplo 2:")
print("T:",T_example[0])
print("Q:",Q_example[0])
for i in range(5):
  print(choose_best_action(0, T_example, Q_example))

Ejemplo 1:
T: [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
Q: [0.6084536  0.94259351 0.58737874 0.06440216 0.23095189 0.16038692
 0.44924275 0.20853705 0.49841264]
None
Ejemplo 2:
T: [-1. -1.  2.  3. -1. -1. -1. -1. -1.]
Q: [0.88008524 0.00433188 0.57100072 0.64769138 0.54292327 0.14334511
 0.78969894 0.33311566 0.00524252]
3
3
3
3
3


Jugador que conoce la mayoría de movimientos iniciales pero mantiene algunas aperturas en segundas/terceras jugadas.

In [ ]:
def choose_advanced_action(state, T, R, board_states):

  board_state = board_states[state]

  #Comprueba victoria o empate directo
  for a in range(T.shape[1]):
    if T[state,a] != -1:
      if R[T[state,a]] != 0 or board_state.count(0) == 1:
        return a

  #Juega al medio
  if board_state[4] == 0:
    return 4

  #Juega a la esquina en segunda jugada
  if board_state == [0,0,0,0,1,0,0,0,0]:
    return np.random.choice([0,2,6,8])

  #Comprueba peligro si el otro jugador tiene 2 fichas en fila y bloqueo
  m = np.asmatrix(np.array(board_state).reshape(3,3))

  # peligros horizontales y verticales
  for i in range(3):
    for j in range(3):
      # 2 fichas iguales en horizontal
      if np.count_nonzero(m[i]) == 2 and  m[i].sum() %2 == 0 and m[i,j] == 0:

        return j + i*3
      # 2 fichas iguales en vertical
      if np.count_nonzero(m.T[j]) == 2 and m.T[j].sum() %2 == 0 and m[i,j] == 0:
        return j + i*3

  # peligros diagonales
  diag1 = m.diagonal()
  diag2 = np.fliplr(m).diagonal()
  for i in range(3):
      if np.count_nonzero(diag1) == 2 and diag1.sum() %2 == 0 and m[i,i] == 0:
        return i*3 + i
      if np.count_nonzero(diag2) == 2 and diag2.sum() %2 == 0 and m[i,2-i] == 0:
        return i*3 + 2-i

  #condiciones especiales
  #Diagonal 121 -> mover lateral
  if board_state == [1,0,0,0,2,0,0,0,1] or board_state == [0,0,1,0,2,0,1,0,0]:
    return np.random.choice([1,3,5,7])
  #Diagonal 112 o simetricas -> mover igual column o fila
  if m.sum() == 4 and (diag1.sum() == 4 or diag2.sum() == 4):
    for i in range(3):
      for j in range(3):
        if m[i,j] == 2:
          return np.random.choice([(2-i)*3 + j, 3*i + (2-j)])

  #aleatorio
  empty_positions = []
  for i in range(T.shape[1]):
    if board_state[i] == 0:
      empty_positions.append(i)
  return np.random.choice(empty_positions)

### Matriz Q

Existen 3 versiones del algoritmo:

- Algoritmo simple que entrena J1 usando turnos del rival
- Algoritmo que entrena J1 saltando los turno del rival
- Algoritmo que entre J1 y J2 saltando turnos del  rival

Algoritmo que entrena la matriz Q a raíz de la ecuación de Bellman. El entrenamiento es sólo para el jugador 1 y las estados en los que mueve el jugador 2 también se retoralimentan de las recompensas, lo implicaría que la IA como jugador 2 ayudaría a ganar al 1.

El factor epsilon-greedy se ha dejado como ecuación de segundo grado para que el 70% prevalezca la fase de exploración y en el último 30% la fase de explotación vaya teniendo más incidencia.

In [ ]:
def train_quality_matriz_simple(T, R, board_states, episodes, v, y, info=False):
  # Matriz de calidad
  Q = np.zeros((T.shape[0],T.shape[1]),dtype=float)

  s = 0 # Estado inicial
  episode = 0 # Inicialización del entrenamiento

  while(episode < episodes):

    # Epsilon-Greedy: Selección de acción
    epsilon = 1 - (episode/episodes)**2

    # Acción aleatoria
    if np.random.rand() < epsilon:
      a=choose_random_action(s, T)
    # Acción según la matriz de calidad (Q)
    else:
      a=choose_best_action(s, T, Q)

    next_s = T[s,a]

    # Actualización de la matriz de calidad
    Qmax = calculate_q_max(next_s, T, Q, True)
    Q[s,a] = (1-v) * Q[s,a] + v * (R[next_s] + y*Qmax)

    # Cambio de estado
    # La segunda condición equivale a completar el tablero si no hubiese recompensas para dicho caso. Ej: r_draw = 0
    if (R[next_s] != 0) or (board_states[next_s].count(0) == 0):
      s = 0
    else:
      s = next_s
    episode += 1

  # INFO
  if info:
    # Comprobación de los estados que no han actualizado su valor en la matriz de calidad y no son finales con recompensa
    Rn0 = len(np.array(R)[np.array(R)!=0])
    print("Estados con recompensas =>", Rn0)
    Qa0 = Q[(Q==0).all(axis=1)].shape[0]
    print("Filas de Q sin entrenar =>", Qa0)
    #Combiene volver a entrenar si algún estado no ha sido entrenado
    print("Índices no entrenados =>", np.setxor1d(np.where((Q == 0).all(axis=1))[0], np.where(np.array(R)!=0)[0]))

  return Q

_episodes = 400000 # Nº de actualizaciones de la matriz Q
_learning_rate = 0.1 # Factor de aprendizaje menor a 1/9 que de mayor peso a la experiencia
_discount_rate = 0.9 # Factor indiferente entre 0.1 y 0.9
_Q = train_quality_matriz_simple(_T, _R, _board_states, _episodes, _learning_rate, _discount_rate, True)

Estados con recompensas => 1370
Filas de Q sin entrenar => 1370
Índices no entrenados => []


Algoritmo que entrena la matriz Q a raíz de la ecuación de Bellman. El entrenamiento es sólo para el jugador 1 omitiendo los estados del segundo jugador.

El factor epsilon-greedy se ha dejado como ecuación de segundo grado para que el 70% prevalezca la fase de exploración y en el último 30% la fase de explotación vaya teniendo más incidencia.

In [ ]:
def train_quality_matriz_1_player(T, R, board_states, episodes, v, y, info=False):
  # Matriz de calidad
  Q=np.zeros((T.shape[0],T.shape[1]),dtype=float)

  s = 0 # Estado inicial
  episode = 0 # Inicialización del entrenamiento

  while(episode < episodes):

    # Epsilon-Greedy: Selección de acción
    epsilon = 1 - (episode/episodes)**2

    # Acciones aleatoria al azar número entero en [0,9] que esté permitida
    if np.random.rand() < epsilon:
      ## Acción aleatoria J1
      a = choose_random_action(s, T)
      current_s=T[s,a]
      if (R[current_s] != 0) or (board_states[current_s].count(0) == 0):
        next_s = current_s
      else:
        # Acción aleatoria J2
        next_s=T[current_s, choose_random_action(current_s, T)]

    # Acciónes buenas
    else:
      ## Mejor acción permitida permitida J1
      a = choose_best_action(s, T, Q)
      current_s=T[s,a]
      if (R[current_s] != 0) or (board_states[current_s].count(0) == 0):
        next_s = current_s
      else:
        # Acción de jugador avanzado J2
        next_s=T[current_s, choose_advanced_action(current_s, T, R, board_states)]

  # Actualización de la matriz de calidad
    Qmax = calculate_q_max(next_s, T, Q, True)
    Q[s,a] = (1-v) * Q[s,a] + v * (R[next_s] + y*Qmax)

    # Cambio de estado
    if (R[next_s] != 0) or (board_states[next_s].count(0) == 0):
      s = 0
    else:
      s = next_s

    episode+=1

  # INFO
  if info:
    # Comprobación de los estados que no han actualizado su valor en la matriz de calidad y no son finales con recompensa
    Rn0 = len(np.array(R)[np.array(R)!=0])
    print("Estados con recompensas =>", Rn0)
    Qa0 = Q[(Q==0).all(axis=1)].shape[0]
    print("Filas de Q sin entrenar =>", Qa0)
    #Combiene volver a entrenar si algún estado no ha sido entrenado
    print("Índices no entrenados =>", np.setxor1d(np.where((Q == 0).all(axis=1))[0], np.where(np.array(R)!=0)[0]))

  return Q

_episodes = 200000 # Nº de actualizaciones de la matriz Q
_learning_rate = 0.1 # Factor de aprendizaje menor a 1/9 que de mayor peso a la experiencia
_discount_rate = 0.9 # Factor indiferente entre 0.1 y 0.9
_Q = train_quality_matriz_1_player(_T, _R, _board_states, _episodes, _learning_rate, _discount_rate, True)

Estados con recompensas => 1370
Filas de Q sin entrenar => 3467
Índices no entrenados => [   1    2    5 ... 5812 5818 5827]


Algoritmo que entrena la matriz Q a raíz de la ecuación de Bellman para ambos jugadores.

El factor epsilon-greedy se ha dejado como ecuación de segundo grado para que el 70% prevalezca la fase de exploración y en el último 30% la fase de explotación vaya teniendo más incidencia.

In [ ]:
def train_quality_matriz_2_players(T, R, board_states, episodes, v, y, info=False):
  # Matriz de calidad
  Q=np.zeros((T.shape[0],T.shape[1]),dtype=float)
  Q1=np.zeros((T.shape[0],T.shape[1]),dtype=float)
  Q2=np.zeros((T.shape[0],T.shape[1]),dtype=float)

  # INFO
  if info:
    # Comprobación de los estados que no han actualizado su valor en la matriz de calidad y no son finales con recompensa
    Rn0 = len(np.array(R)[np.array(R)!=0])
    print("Estados con recompensas =>", Rn0)
    Qa0 = Q[(Q==0).all(axis=1)].shape[0]
    print("Filas de Q sin entrenar al inicio =>", Qa0)
    #Combiene volver a entrenar si algún estado no ha sido entrenado
    print("Índices no entrenados al inicio =>", np.setxor1d(np.where((Q == 0).all(axis=1))[0], np.where(np.array(R)!=0)[0]))

  episodes = int(episodes/2)

  #-------------------------#
  # ENTRENAMIENTO JUGADOR 1 #
  #-------------------------#

  s = 0 # Estado inicial
  episode = 0 # Inicialización del entrenamiento

  while(episode < episodes):

    # Epsilon-Greedy: Selección de acción
    epsilon = 1 - (episode/episodes)**2

    # Acciones aleatoria al azar número entero en [0,9] que esté permitida
    if np.random.rand() < epsilon:
      ## Acción aleatoria J1
      a = choose_random_action(s, T)
      current_s=T[s,a]
      if (R[current_s] != 0) or (board_states[current_s].count(0) == 0):
        next_s = current_s
      else:
        # Acción aleatoria J2
        next_s=T[current_s, choose_random_action(current_s, T)]

    # Acciónes buenas
    else:
      ## Mejor acción permitida permitida J1
      a = choose_best_action(s, T, Q1)
      current_s=T[s,a]
      if (R[current_s] != 0) or (board_states[current_s].count(0) == 0):
        next_s = current_s
      else:
        # Acción de jugador avanzado J2
        next_s=T[current_s, choose_advanced_action(current_s, T, R, board_states)]

    # Actualización de Q con Q1
    Qmax = calculate_q_max(next_s, T, Q1, True)
    Q1[s,a] = (1-v) * Q1[s,a] + v * (R[next_s] + y*Qmax)
    Q[s,a] = Q1[s,a]

    # Transición de estado
    if (R[next_s] != 0) or (board_states[next_s].count(0) == 0):
      s = 0
    else:
      s = next_s

    episode+=1

  # INFO
  if info:
    # Comprobación de los estados que no han actualizado su valor en la matriz de calidad y no son finales con recompensa
    Qa0 = Q[(Q==0).all(axis=1)].shape[0]
    print("Filas de Q sin entrenar a mitad =>", Qa0)
    #Combiene volver a entrenar si algún estado no ha sido entrenado
    print("Índices no entrenados a mitad=>", np.setxor1d(np.where((Q == 0).all(axis=1))[0], np.where(np.array(R)!=0)[0]))

  #-------------------------#
  # ENTRENAMIENTO JUGADOR 2 #
  #-------------------------#

  s = 0 # Estado inicial
  episode = 0 # Inicialización del entrenamiento
  R = [i*(-1) for i in R]

  while(episode < episodes):

    # Epsilon-Greedy: Selección de acción
    epsilon = 1 - (episode/episodes)**2

    # Acciones aleatoria al azar número entero en [0,9] que esté permitida
    if np.random.rand() < epsilon:

      #Primera acción del jugador 1 en la partida
      if s==0:
        s=T[s, choose_random_action(s, T)]

      ## Acción aleatoria J2
      a = choose_random_action(s, T)
      current_s=T[s,a]
      if (R[current_s] != 0) or (board_states[current_s].count(0) == 0):
        next_s = current_s
      else:
        # Acción aleatoria J1
        next_s=T[current_s, choose_random_action(current_s, T)]

    # Acciónes buenas
    else:
      #Primera acción del jugador avanzado en la partida
      if s==0:
        s=T[s, choose_advanced_action(s, T, R, board_states)]

      ## Mejor acción permitida permitida J2
      a = choose_best_action(s, T, Q2)
      current_s=T[s,a]
      if (R[current_s] != 0) or (board_states[current_s].count(0) == 0):
        next_s = current_s
      else:
        # Acción de jugador avanzado J1
        next_s=T[current_s, choose_advanced_action(current_s, T, R, board_states)]

    # Actualización de Q con Q2
    Qmax = calculate_q_max(next_s, T, Q2, True)
    Q2[s,a] = (1-v) * Q2[s,a] + v * (R[next_s] + y*Qmax)
    Q[s,a] = Q2[s,a]

    # Transición de estado
    if (R[next_s] != 0) or (board_states[next_s].count(0) == 0):
      s = 0
    else:
      s = next_s

    episode+=1

  # INFO
  if info:
    # Comprobación de los estados que no han actualizado su valor en la matriz de calidad y no son finales con recompensa
    Qa0 = Q[(Q==0).all(axis=1)].shape[0]
    print("Filas de Q sin entrenar al final =>", Qa0)
    #Combiene volver a entrenar si algún estado no ha sido entrenado
    print("Índices no entrenados al final =>", np.setxor1d(np.where((Q == 0).all(axis=1))[0], np.where(np.array(R)!=0)[0]))

  return Q

_episodes = 200000 # Nº de actualizaciones de la matriz Q
_learning_rate = 0.1 # Factor de aprendizaje menor a 1/9 que de mayor peso a la experiencia
_discount_rate = 0.9 # Factor indiferente entre 0.1 y 0.9
_Q = train_quality_matriz_2_players(_T, _R, _board_states, _episodes, _learning_rate, _discount_rate, True)

Estados con recompensas => 1370
Filas de Q sin entrenar al inicio => 5890
Índices no entrenados al inicio => [   0    1    2 ... 5828 5836 5843]
Filas de Q sin entrenar a mitad => 3467
Índices no entrenados a mitad=> [   1    2    5 ... 5812 5818 5827]
Filas de Q sin entrenar al final => 1370
Índices no entrenados al final => []


### Funciones de comprobación

In [ ]:
#Función para ver el tablero y matrices de un estado
def check_state(state, T, Q, R, board_states):
  df = pd.DataFrame([['x' if i == 1 else 'o' if i == 2 else "-" for i in board_states[state]], T[state], Q[state]],
                    columns=["Acción "+str(i) for i in list(range(1,10))],
                    index=['Tablero', 'T', 'Q'])

  df_style = df.style.apply(lambda row: ['color: green' if x == df.loc['Q', :].replace(0,pd.NA).dropna().max(axis=0) else '' for x in row], subset=pd.IndexSlice[['Q'], :])

  display(df_style)

  print("Recompensa:", R[state])

check_state(5827, _T, _Q, _R, _board_states)

,Acción 1,Acción 2,Acción 3,Acción 4,Acción 5,Acción 6,Acción 7,Acción 8,Acción 9
Tablero,o,o,x,o,x,x,-,x,-
T,-1,-1,-1,-1,-1,-1,5835,-1,5828
Q,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Recompensa: 0


In [ ]:
#Comprobar Estado destino de jugar
def check_previous_states(state_checked, T, Q, R, board_states, dict_state_value, dict_value_state):

  print("Estado:")
  board_state = board_states[state_checked]
  board_value = dict_state_value[state_checked]
  print(f"{state_checked} (V {board_value}) => {board_state} => {T[state_checked]} => {Q[state_checked].round(2)} => R {R[state_checked]}")

  coin = 1 if board_state.count(1) - board_state.count(2) == 1 else 2
  possible_indexes = np.where(np.array(board_states[state_checked]) == coin)[0]
  print(f"\nJugadas anteriores -> índices {possible_indexes}")
  for i in possible_indexes:
    previous_state = board_value - coin*3**(8-i)
    previous_index = dict_value_state[previous_state]
    print(f"{previous_index} (V {previous_state}) => {board_states[previous_index]} => {T[previous_index]} => {Q[previous_index].round(2)}")

check_previous_states(1702, _T, _Q, _R, _board_states, _dict_state_value, _dict_value_state)

Estado:
1702 (V 5062) => [0, 2, 0, 2, 2, 1, 1, 1, 1] => [-1 -1 -1 -1 -1 -1 -1 -1 -1] => [0. 0. 0. 0. 0. 0. 0. 0. 0.] => R 10

Jugadas anteriores -> índices [5 6 7 8]
1698 (V 5035) => [0, 2, 0, 2, 2, 0, 1, 1, 1] => [-1 -1 -1 -1 -1 -1 -1 -1 -1] => [0. 0. 0. 0. 0. 0. 0. 0. 0.]
1699 (V 5053) => [0, 2, 0, 2, 2, 1, 0, 1, 1] => [3682   -1 1957   -1   -1   -1 1702   -1   -1] => [ 7.85  0.    9.66  0.    0.    0.   10.    0.    0.  ]
1700 (V 5059) => [0, 2, 0, 2, 2, 1, 1, 0, 1] => [3684   -1 1959   -1   -1   -1   -1 1702   -1] => [2.93 0.   9.58 0.   0.   0.   0.   5.22 0.  ]
1701 (V 5061) => [0, 2, 0, 2, 2, 1, 1, 1, 0] => [3685   -1 1960   -1   -1   -1   -1   -1 1702] => [ 7.47  0.    6.    0.    0.    0.    0.    0.   10.  ]


## Funciones de Juego

### Funciones partida con interfaz

In [ ]:
# Tablero de juego con botones 3x3
def create_board(on_square_click):
  global buttons
  buttons =  []
  buttons_margin = ['0 0 5px -5px', '0 0 0 0', '0 0 0 5px']
  for i in range(9):
    button = widgets.Button(description='', button_style='',
            layout=widgets.Layout(width= '80px', height= '80px'))
    button.layout.margin = buttons_margin[i%3]
    button.on_click(on_square_click)
    buttons.append(button)

    board = widgets.GridBox(buttons,
                            layout=widgets.Layout(grid_template_columns="repeat(3, 80px)"))
  display(board)

#Funcion para modificar el tablero de juego
def choose_square(button, description, style):
  button.description = description
  button.button_style = style
  button.disabled = True

def disable_squares(buttons):
  for button in buttons:
    if button.description == '':
      button.disabled = True

def enable_squares(buttons):
  for button in buttons:
    if button.description == '':
      button.disabled = False

def reset_board (buttons):
  current_state = 0
  for button in buttons:
    button.description = ''
    button.button_style = ""
    button.disabled = False

In [ ]:
# Función que comprueba estado final
def finish_turn(player, button):
  global current_state
  global board_states
  global R
  global rewards

  if player == 1:
    print("Tú: ", current_state)
  else:
    print("IA: ", current_state)

  # Se marca la casilla elegida
  coin = 'O' if player == 1 else 'X' # "Jugador 'O' - IA 'X"
  # Jugado con victoria de un jugador
  if R[current_state] == rewards[0] or R[current_state] == rewards[1]:
    style = 'success' if player == 1 else 'danger' # "Jugador 'verde' - IA 'rojo"
  # Jugado sin victoria
  else:
    style = 'info' if player == 1 else 'warning' # "Jugador 'azul' - IA 'naranja"
  choose_square(button, coin, style)

  # Turno final
  if (R[current_state] != 0) or (board_states[current_state].count(0) == 0):
    disable_squares(buttons)
    if R[current_state] == rewards[0]:
      print("Jugador 1 gana")
    elif R[current_state] == rewards[1]:
      print("Jugador 2 gana")
    else:
      print("Empate")
  #Turno normal
  else:
    if player == 1:
      disable_squares(buttons)
      launch_ia()
    else:
      enable_squares(buttons)

# Función para manejar el clic en el botón
def push_square(button):
  global current_state
  global buttons
  indice = buttons.index(button)
  current_state = T[current_state, indice]
  finish_turn(1, button)

# Función para que la IA juegue
def launch_ia():
  global current_state
  global buttons
  # Obtener botones disponibles
  buttons_availables = [button for button in buttons if button.description == '']
  if buttons_availables:
    if (board_states[current_state].count(1) - board_states[current_state].count(2)) == 0:
      index = choose_best_action(current_state, T, Q)
    else:
      index = choose_best_action(current_state, T, Q)
    current_state = T[current_state, index]
    button = buttons[index]
    finish_turn(2, button)

### Funciones de partidas automáticas

In [ ]:
# Función para que mueva la IA
def move_IA(current_state, T, Q):
  if (board_states[current_state].count(1) - board_states[current_state].count(2)) == 0:
    index = choose_best_action(current_state, T, Q)
  else:
    index = choose_best_action(current_state, T, Q)
  current_state = T[current_state, index]

  return current_state
# Función para movimiento aleatorio
def move_random(current_state, T):
  index = choose_random_action(current_state, T)
  current_state = T[current_state, index]
  return current_state
# Función para movimiento avanzado
def move_advanced(current_state, T):
  global R
  global board_states
  index = choose_advanced_action(current_state, T, R, board_states)
  current_state = T[current_state, index]
  return current_state

#Algoritmo de partida automático donde en cada turno mueve un jugador
def start_ia_vs_rival(T,Q, IA_play_first=True, is_rival_random=True):

  n_wins_j1 = 0
  n_wins_j2 = 0
  n_draws = 0

  i_aux = 0 if IA_play_first else 1

  for i in range(10000):

    current_state = 0

    while R[current_state] == 0 and board_states[current_state].count(0) != 0:
      if (i+i_aux)%2 == 0:
        current_state = move_IA(current_state, T, Q)
      else:
        if is_rival_random:
          current_state = move_random(current_state, T)
        else:
          current_state = move_advanced(current_state, T)

    if R[current_state] == rewards[0]:
      n_wins_j1+=1
    if R[current_state] == rewards[1]:
      n_wins_j2+=1
    else:
      n_draws+=1

  print("Victorias J1: ",n_wins_j1 )
  print("Victorias J2: ",n_wins_j2 )
  print("Empates:      ",n_draws )

# Juego

## Variables e hiperparámetros

In [ ]:
# Recompensas de victoria/derrota/empate
rewards = [10, -10, 1]
n_episodes = 300000 #epidios de entrenamiento
v = 0.1 #Factor de aprendizaje menor a 1/9 que de mayor peso a la experiencia
y = 0.7 #Factor de descuento indiferente entre 0.1 y 0.9

board_states, R = create_board_and_rewards_vectors(rewards, False)
dict_state_value, dict_value_state = create_dicts_state_value(board_states, False)
T = create_transition_matrix(board_states, R, dict_state_value, dict_value_state, False)

#Elegir algoritmo de entrenamiento
#Q = train_quality_matriz_simple(T, R, board_states, n_episodes, v, y, True)
#Q = train_quality_matriz_1_player(T, R, board_states, n_episodes, v, y, True)
Q = train_quality_matriz_2_players(T, R, board_states, n_episodes, v, y, True)

Estados con recompensas => 1370
Filas de Q sin entrenar al inicio => 5890
Índices no entrenados al inicio => [   0    1    2 ... 5828 5836 5843]
Filas de Q sin entrenar a mitad => 3467
Índices no entrenados a mitad=> [   1    2    5 ... 5812 5818 5827]
Filas de Q sin entrenar al final => 1370
Índices no entrenados al final => []


## Tablero

Interfaz para jugar partidas

In [ ]:
IA_player1 = True

#Variable de estado
current_state = 0
buttons = []
# Mostrar el tablero
create_board(push_square)

if IA_player1:
  launch_ia()

## Partidas automáticas



Estadísticas de victorias contra ciertos rivales.

In [ ]:
#IA vs jugador random
start_ia_vs_rival(T,Q, IA_play_first=True, is_rival_random=True)

Victorias J1:  7971
Victorias J2:  1381
Empates:       8619


In [ ]:
#IA vs jugador avanzado
start_ia_vs_rival(T,Q, IA_play_first=True, is_rival_random=False)

Victorias J1:  5000
Victorias J2:  301
Empates:       9699
